In [1]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [2]:
# 示例 7-18 生成第 n 个斐波那契数，递归耗时方法

@clock 
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

if __name__ == '__main__':
    print(fibonacci(5))

[0.00000095s] fibonacci(1) -> 1 
[0.00000048s] fibonacci(0) -> 0 
[0.00000048s] fibonacci(1) -> 1 
[0.00084257s] fibonacci(2) -> 1 
[0.00173974s] fibonacci(3) -> 2 
[0.00000024s] fibonacci(0) -> 0 
[0.00000024s] fibonacci(1) -> 1 
[0.00013947s] fibonacci(2) -> 1 
[0.00000024s] fibonacci(1) -> 1 
[0.00000048s] fibonacci(0) -> 0 
[0.00000024s] fibonacci(1) -> 1 
[0.00014901s] fibonacci(2) -> 1 
[0.00035453s] fibonacci(3) -> 2 
[0.00063586s] fibonacci(4) -> 3 
[0.00251579s] fibonacci(5) -> 5 
5


In [3]:
# 示例 7-19 使用缓存实现，速度更快
from functools import lru_cache

@lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

if __name__ == '__main__':
    print(fibonacci(5))

[0.00000048s] fibonacci(1) -> 1 
[0.00000024s] fibonacci(0) -> 0 
[0.00008154s] fibonacci(2) -> 1 
[0.00100470s] fibonacci(3) -> 2 
[0.00000048s] fibonacci(4) -> 3 
[0.00118637s] fibonacci(5) -> 5 
5


In [11]:
# 7.8.2 单分派泛函数

import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

htmlize('<h1>你好</h1>')

'<pre>&#x27;&lt;h1&gt;你好&lt;/h1&gt;&#x27;</pre>'

In [12]:
# 示例 7-20 生成 HTML 的 htmlize 的函数，调整了几种对象的输出
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [13]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [16]:
htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

In [14]:
htmlize(42)

'<pre>42</pre>'

In [15]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [29]:
# singledispatch 创建一个自定义的
# htmlize.register 装饰器，把多个函数绑在一起组成一个泛函数

from functools import singledispatch
from collections import abc
import numbers
import html


@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    # {0} 输出第一个 数字
    # {0:x} 输出 第一个数字的 十六进制 
    # 类似的还有 {:b} {:d} {:o} 二进制 十进制 八进制
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [19]:
htmlize('hello world\n')

'<p>hello world<br>\n</p>'

In [30]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [23]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [31]:
# 7.10 参数化装饰器
# 示例 7-22 

registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x7ff2e151b1e0>)
running main()
registry -> [<function f1 at 0x7ff2e151b1e0>]
running f1()


In [ ]:
# 7.10.1 一个参数化的注册装饰器
"""
为了便于启用或禁用 register 执行的函数注册功能，我们为它提供一 个可选的 active 参数，
设为 False 时，不注册被装饰的函数。
实现方 式参见示例 7-23。从概念上看，这个新的 register 函数不是装饰器， 而是装饰器工厂函数。
调用它会返回真正的装饰器，这才是应用到目标 函数上的装饰器。
"""

In [36]:
# 示例 7-23 为了接受参数，新的 register 装饰器必须作为函数调用

registry = set()

def register(active=True):
    def decorate(func):
        print(f"running register(active={active}-->decorate({func}))")
        
        if active:
            registry.add(func)
        else:
            registry.discard(func)
            
        return func
    return decorate

@register(active=False)
def f1():
    print("running f1()")
    
@register()
def f2():
    print("running f2()")
    
def f3():
    print("running f3()")

running register(active=False-->decorate(<function f1 at 0x7ff2e151b400>))
running register(active=True-->decorate(<function f2 at 0x7ff2e14e1d90>))


In [37]:
registry

{<function __main__.f2()>}

In [38]:
register()(f3)

running register(active=True-->decorate(<function f3 at 0x7ff2e1612e18>))


<function __main__.f3()>

In [39]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [40]:
register(active=False)(f2)

running register(active=False-->decorate(<function f2 at 0x7ff2e14e1d90>))


<function __main__.f2()>

In [41]:
registry

{<function __main__.f3()>}

In [54]:
# 7.10.2 参数化clock装饰器
"""
本节再次探讨 clock 装饰器，为它添加一个功能:让用户传入一个格
式字符串，控制被装饰函数的输出。参见示例 7-25。
"""

# 示例 7-25 clockdeco_param.py 模块，参数化 clock 装饰器

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) => {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

if __name__ == '__main__':
    @clock
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)


In [ ]:
"""
示例 7-26 和示例 7-27 是另外两个模块，它们使用了 clockdeco_param 模块中的新功能，随后是两个模块输出的结果。
"""

In [52]:
# 示例 7-26 clockdeco_param_demo1.py

import time
# from clockdeco_param import clock

@clock('{name}: {elapsed}s') 
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3): 
    snooze(.123)

snooze: 0.12383365631103516s
snooze: 0.12324905395507812s
snooze: 0.12314653396606445s


In [55]:
# 示例 7-27 clockdeco_param_demo2.py

import time
# from clockdeco_param import clock

@clock('{name}({args}) dt={elapsed:0.3f}s') 
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3): 
    snooze(.123)

snooze(0.123) dt=0.124s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
